# Firearms Import Control Cross-Referencing and Matching System

In [ ]:
# Import libraries to be used
import pandas as pd
import re

# DB A - DOCUMENTATION - df_doc

Import documentation database as df_doc_original, with index = column '_record_id' and forcing SerialNumber column to 'str' dtype (needed to avoid losing '0' on the left and including alphanumeric patterns).

In [ ]:
# Import
df_doc_original = pd.read_excel('documentation_202404reportv2.xlsx',
                                 dtype={'SerialNumber': str},
                                index_col='_record_id')
df_doc_original

In [7]:
# Check information about dtypes and null values
# 'SerialNumber must' be 'str' (object)
# 'Date Documented' must be datetime

df_doc_original.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12462 entries, dbc09958-9d76-4a53-9675-b35e27c0031b to 95ee9174-1d1a-4119-bbf3-59a7e7d44e1a
Data columns (total 30 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   source                                         12462 non-null  object        
 1   status                                         0 non-null      float64       
 2   MATCH YES / NO                                 648 non-null    object        
 3   lambarka_warshadda_manufacturer_serial_number  12305 non-null  object        
 4   lenght                                         0 non-null      float64       
 5   calaamada_dowladda_government_mark             12420 non-null  object        
 6   taariikh_date                                  12462 non-null  datetime64[ns]
 7   security_force                                 12462 non-null  object        


In [8]:
# Create a copy of df_doc to preserve the original
df_doc = df_doc_original.copy()

In [5]:
# Create function to clean all column names
def clean_column_name(col):
    """
    Clean a column name by converting it to lowercase, replacing
    non-alphanumeric characters with '_', and removing consecutive underscores.

    Args:
        col (str): The column name to be cleaned.

    Returns:
        str: The cleaned column name.
    """
    col = col.lower()  # Convert to lowercase
    col = re.sub(r'\W+', '_', col)  # Replace non-alphanumeric characters with '_'
    col = col.replace('___', '_')
    return col

In [6]:
# Apply clean_column_name function and rename df_doc columns
df_doc.columns = [clean_column_name(col) for col in df_doc.columns]
df_doc.columns

Index(['source', 'status', 'match_yes_no',
       'lambarka_warshadda_manufacturer_serial_number', 'lenght',
       'calaamada_dowladda_government_mark', 'taariikh_date', 'security_force',
       'security_force2', 'region', 'district',
       'location_police_station_unitunit', 'gobolka_degmo_region_district',
       '_latitude', '_longitude', 'other_unique_identifying_mark',
       'generic_category_1', 'generic_category_2', 'generic_category_3',
       'model_system_designation', 'calibre', 'year_of_manufacture',
       'manufacturing_country', 'qa_note', 'end_user_indicated',
       'year_imported', 'exporting_state', 'un_note_number', 'accept_reject',
       'qa_by'],
      dtype='object')

In [7]:
# Rename and simplify specific columns that will be intensively used in the cross-reference effort
df_doc = df_doc.rename(columns={'calaamada_dowladda_government_mark': 'gvtmk',
                                'lambarka_warshadda_manufacturer_serial_number': 'sn',
                                'taariikh_date': 'date_doc',
                                'generic_category_2': 'category',
                                'model_system_designation': 'model',
                                'year_of_manufacture': 'yom',
                                'manufacturing_country': 'mfr_country'})

df_doc.columns

Index(['source', 'status', 'match_yes_no', 'sn', 'lenght', 'gvtmk', 'date_doc',
       'security_force', 'security_force2', 'region', 'district',
       'location_police_station_unitunit', 'gobolka_degmo_region_district',
       '_latitude', '_longitude', 'other_unique_identifying_mark',
       'generic_category_1', 'category', 'generic_category_3', 'model',
       'calibre', 'yom', 'mfr_country', 'qa_note', 'end_user_indicated',
       'year_imported', 'exporting_state', 'un_note_number', 'accept_reject',
       'qa_by'],
      dtype='object')

In [8]:
# Create a DF's copy and duplicate specified columns to preserve original data (sufix '_original')
df_doc = df_doc.copy()
columns_to_copy = ['sn', 'gvtmk', 'category', 'model',
                   'calibre', 'date_doc', 'yom', 'mfr_country']
for column in columns_to_copy:
    df_doc[column + '_original'] = df_doc[column]
df_doc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12462 entries, dbc09958-9d76-4a53-9675-b35e27c0031b to 95ee9174-1d1a-4119-bbf3-59a7e7d44e1a
Data columns (total 38 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   source                            12462 non-null  object        
 1   status                            0 non-null      float64       
 2   match_yes_no                      648 non-null    object        
 3   sn                                12305 non-null  object        
 4   lenght                            0 non-null      float64       
 5   gvtmk                             12420 non-null  object        
 6   date_doc                          12462 non-null  datetime64[ns]
 7   security_force                    12462 non-null  object        
 8   security_force2                   12462 non-null  object        
 9   region                            12448 non-null  objec

In [9]:
# Replace NaN values with 'NA'
columns_na = ['sn', 'category', 'model', 'calibre', 'gvtmk',
              'date_doc', 'mfr_country']
for column in columns_na:
    df_doc[column] = df_doc[column].fillna('NA')

In [10]:
# Replace null values with 0
columns_to_zero = ['yom', 'yom_original']
for column in columns_to_zero:
    df_doc[column] = df_doc[column].fillna(0)

In [11]:
# Force columns ['yom', 'yom_original'] to int
columns_to_int = ['yom', 'yom_original']
for column in columns_to_int:
    df_doc[column] = df_doc[column].astype(int)

# Check
df_doc['yom'].dtype

dtype('int64')

In [12]:
# Force columns ['sn', 'gvtmk', 'category', 'model', 'calibre', 'mfr_country'] to 'str' (object)
columns_to_str = ['sn', 'gvtmk', 'category', 'model', 'calibre','mfr_country']
for column in columns_to_str:
    df_doc[column] = df_doc[column].astype(str)

# Check
df_doc['sn'].dtype

dtype('O')

**Harmonizing category names**

Acoording to the following table:

| Original                           | Modified for Use in Match |
|------------------------------------|----------------------------|
| Assault rifle                      | assaultrifle               |
| Medium machine gun (MMG)           | mg                         |
| Self-loading pistol                | pistol                     |
| Shoulder-fired rocket launchers    | rpg                        |
| Light machine gun (LMG)            | mg                         |
| Sniper rifles                      | dmr                        |
| Heavy machine gun (HMG)            | hmg                        |
| Anti-materiel rifles (AMR)         | amr                        |
| Self-loading rifle or carbine      | rifle                      |

In [13]:
# Check
df_doc['category'].value_counts()

category
Assault rifle                      11246
Self-loading pistol                  512
Medium machine gun (MMG)             371
Shoulder-fired rocket launchers      171
Light machine gun (LMG)               88
Sniper rifles                         43
Heavy machine gun (HMG)               19
Assault Rifle                          8
Self-loading rifle or carbine          2
Anti-materiel rifles (AMR)             2
Name: count, dtype: int64

In [14]:
# Create a mapping dictionary
doc_category_mapping = {
    "Assault rifle": "assaultrifle",
    "Assault Rifle": "assaultrifle",
    "Medium machine gun (MMG)": "mg",
    "Self-loading pistol": "pistol",
    "Shoulder-fired rocket launchers": "rpg",
    "Light machine gun (LMG)": "mg",
    "Sniper rifles": "dmr",
    "Heavy machine gun (HMG)": "hmg",
    "Anti-materiel rifles (AMR)": "amr",
    "Self-loading rifle or carbine": "rifle"
}

# Replace the 'category' column values based on the mapping
df_doc['category'] = df_doc['category'].map(doc_category_mapping).fillna(df_doc['category'])

# Display the DataFrame to verify the changes
df_doc['category'].value_counts()


category
assaultrifle    11254
pistol            512
mg                459
rpg               171
dmr                43
hmg                19
rifle               2
amr                 2
Name: count, dtype: int64

In [15]:
# Change columns ['sn', 'gvtmk', 'mfr_country'] data to upper case
columns_to_upper = ['sn', 'gvtmk', 'mfr_country']
for column in columns_to_upper:
    df_doc[column] = df_doc[column].str.upper()

In [ ]:
# Change columns ['category', 'model', 'calibre'] data to lower case
columns_to_lower = ['category', 'model', 'calibre']
for column in columns_to_lower:
    df_doc[column] = df_doc[column].str.lower()
df_doc.head(3)

In [17]:
# Count spaces
count_spaces_per_column = df_doc.map(lambda x: ' ' in str(x)).sum()
print("Number of cells with spaces in each column:")
print(count_spaces_per_column)

Number of cells with spaces in each column:
source                                  0
status                                  0
match_yes_no                            0
sn                                   1053
lenght                                  0
gvtmk                               11745
date_doc                            12462
security_force                      11941
security_force2                         0
region                               1679
district                             1599
location_police_station_unitunit    10638
gobolka_degmo_region_district        3067
_latitude                               0
_longitude                              0
other_unique_identifying_mark         787
generic_category_1                  11950
category                                0
generic_category_3                    175
model                               10633
calibre                             12461
yom                                     0
mfr_country                     

In [18]:
# Delete spaces, '-' and '.' in selected columns
columns_to_edit = ['sn', 'category', 'model', 'calibre', 'gvtmk']

for column in columns_to_edit:
    df_doc[column] = df_doc[column].str.replace(' ', '', regex=True)  # delete spaces
    df_doc[column] = df_doc[column].str.replace('-', '', regex=True) # delete '-'

# Delete '.' in ['model'] column
columns_to_edit_point = ['model']
for column in columns_to_edit_point:
    df_doc[column] = df_doc[column].str.replace('.', '') # delete '.'

# Delete '.' in ['sn'] column
df_doc['sn'] = df_doc['sn'].str.replace('.', '') # delete '.'

# Fill NaN values in ['yom'] and force dtype to int
df_doc['yom'] = df_doc['yom'].fillna(0).astype(int)
df_doc.head(3)

,source,status,match_yes_no,sn,lenght,gvtmk,date_doc,security_force,security_force2,region,...,accept_reject,qa_by,sn_original,gvtmk_original,category_original,model_original,calibre_original,date_doc_original,yom_original,mfr_country_original
_record_id,,,,,,,,,,,,,,,,,,,,,
dbc09958-9d76-4a53-9675-b35e27c0031b,Fulcrum-202404-SNA,NaN,NaN,5100631,NaN,SOXDS2024,2024-03-14,Ciidanka Xoogga Dalka Soomaaliyeed,SNA,Banadir,...,1,BL,5100631,SO XDS 2024,Shoulder-fired rocket launchers,Type 69-1,40 mm,2024-03-14,0,CHN People's Republic of China
7e0bfa48-126a-4cba-a46e-1d379fe1ff8b,Fulcrum-202404-SNA,NaN,NaN,5100632,NaN,SOXDS2024,2024-03-14,Ciidanka Xoogga Dalka Soomaaliyeed,SNA,Banadir,...,1,BL,5100632,SO XDS 2024,Shoulder-fired rocket launchers,Type 69-1,40 mm,2024-03-14,0,CHN People's Republic of China
61850fbf-3c0d-424e-a8d6-bf7e20ef80cd,Fulcrum-202404-SNA,NaN,NaN,5100549,NaN,SOXDS2024,2024-03-13,Ciidanka Xoogga Dalka Soomaaliyeed,SNA,Banadir,...,1,BL,5100549,SO XDS 2024,Shoulder-fired rocket launchers,Type 69-1,40 mm,2024-03-13,0,CHN People's Republic of China


In [19]:
# Check
df_doc['yom'].value_counts().sort_index()

yom
0       5479
1944       1
1950       2
1951      32
1952      10
1953      66
1954      39
1955      12
1956       1
1957       1
1958       7
1959      11
1960      65
1961      38
1962      59
1963      95
1964      60
1965       7
1966       6
1967       7
1968      19
1969      36
1970      17
1971      15
1972      24
1973      13
1974      45
1975      23
1976      26
1977      42
1978      16
1979       2
1980       9
1981       2
1982       8
1983      23
1984      10
1985      23
1986      35
1987      12
1988      13
1989       3
1994       5
1995       1
1996       1
1997       1
1998       5
1999       2
2000       6
2015      43
2016      24
2017      30
2018    5413
2019     162
2020      74
2021      49
2022     199
2023      33
Name: count, dtype: int64

**Change data in column yom**

Rules

- if in yom_original
    - model is 'type561' and
    - sn has only numbers (digits) and lenght 8 and the 2 first digits as column sn[0,1] in the table below:
    - then yom entrance will change accordingly with column n yom in the table bellow.

| sn[0,1] | n yom|
|---------|------|
| 68      | 2023 |
| 67      | 2022 |
| 66      | 2021 |
| 65      | 2020 |
| 64      | 2019 |
| 63      | 2018 |
| 62      | 2017 |
| 61      | 2016 |
| 60      | 2015 |
| 59      | 2014 |
| 58      | 2013 |
| 57      | 2012 |
| 56      | 2011 |
| 55      | 2010 |
| 54      | 2009 |

In [20]:
# Dictionary to map the first two digits of 'sn' to the new value of 'yom'
yom_mapping = {
    '68': 2023,
    '67': 2022,
    '66': 2021,
    '65': 2020,
    '64': 2019,
    '63': 2018,
    '62': 2017,
    '61': 2016,
    '60': 2015,
    '59': 2014,
    '58': 2013,
    '57': 2012,
    '56': 2011,
    '55': 2010,
    '54': 2009,
}

# Apply change rules in each row
for index, row in df_doc.iterrows():
    if row['model'] == 'type561' and row['sn'].isdigit() and len(row['sn']) == 8:
        sn_prefix = row['sn'][:2]  #Gets the first two digits of 'sn'
        if sn_prefix in yom_mapping:
            #Updates 'yom' only for the current row
            df_doc.at[index, 'yom'] = yom_mapping[sn_prefix]

In [22]:
# Check
df_doc['yom'].value_counts().sort_index()

yom
0       4999
1944       1
1950       2
1951      32
1952      10
1953      66
1954      39
1955      12
1956       1
1957       1
1958       7
1959      11
1960      65
1961      38
1962      59
1963      95
1964      60
1965       7
1966       6
1967       7
1968      19
1969      36
1970      17
1971      15
1972      24
1973      13
1974      45
1975      23
1976      26
1977      42
1978      16
1979       2
1980       9
1981       2
1982       8
1983      23
1984      10
1985      23
1986      35
1987      12
1988      13
1989       3
1994       5
1995       1
1996       1
1997       1
1998       5
1999       2
2000       6
2015     297
2016       3
2017      35
2018    5737
2019     155
2020     123
2021      90
2022      35
2023      32
Name: count, dtype: int64

In [23]:
# Create a new colum 'year_doc' based on info from column 'date_doc'.
# This is needed for quality checks that will avoid that we match consignments received after the date that weapon was documented.

# Change 'date_doc' dtype to 'date'
df_doc['date_doc'] = pd.to_datetime(df_doc['date_doc'])

# Create a new colum 'year_doc' based on info from column 'date_doc'
df_doc['year_doc'] = df_doc['date_doc'].dt.year

df_doc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12462 entries, dbc09958-9d76-4a53-9675-b35e27c0031b to 95ee9174-1d1a-4119-bbf3-59a7e7d44e1a
Data columns (total 39 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   source                            12462 non-null  object        
 1   status                            0 non-null      float64       
 2   match_yes_no                      648 non-null    object        
 3   sn                                12462 non-null  object        
 4   lenght                            0 non-null      float64       
 5   gvtmk                             12462 non-null  object        
 6   date_doc                          12462 non-null  datetime64[ns]
 7   security_force                    12462 non-null  object        
 8   security_force2                   12462 non-null  object        
 9   region                            12448 non-null  objec

In [24]:
# Reorganize columns
df_doc = df_doc[[
    'sn', 'gvtmk', 'category', 'model', 'mfr_country', 'calibre', 'year_doc',
    'yom', 'date_doc'] + [
        col for col in df_doc.columns if col not in [
            'sn', 'gvtmk', 'category', 'model', 'mfr_country',
            'calibre', 'year_doc', 'yom', 'date_doc']]]

In [ ]:
# Final check
df_doc.head(3)

# DB B - IMPORT - df_imp

Import the Excel file and allow Pandas to assign a default numeric index.

Unlike the documentation database, which already comes from Fulcrum with a good index or unique identifier (the record_id), this database does not have such data to use as an index and unique identifier. Therefore, it is necessary to create this key.

In [ ]:
# Loade the import database
# Specify only the tab/sheet we need to load/import
sheet_name = '3. ALL WEAPONS'
df_imp_original = pd.read_excel(
    '---UNSC NOTIFICATIONS DATABASE- 27k April2024.xlsx',
    sheet_name=sheet_name, dtype={'SERIAL NUMBER': str})
#Force SN as str to avoid loosing 0s on the left

# Rename the index to start with 'IMP'
df_imp_original.index = 'IMP' + (df_imp_original.index + 1).astype(str)
df_imp_original.head(3)

In [27]:
# Copy the original df_imp
df_imp = df_imp_original.copy()
df_imp.columns

Index(['TYPE', 'MANUFACTURING COUNTRY', 'MANUFACTURER', 'YEAR OF MANUFACTURE',
       'CATEGORY', 'MODEL_orig', 'Model2', 'CALIBRE', 'SERIAL NUMBER',
       'GOVERNMENT USER MARK', 'WEAPON NUMBER', 'BATCH / LOT NUMBER',
       'QUANTITY IN BOX / CRATE', 'EXPIRY DATE ', 'BOX NUMBER', 'OUT OF TOTAL',
       'END USER INDICATED', 'NOTIFICATION FILE NAME', 'YEAR',
       'EXPORTING STATE', 'UN NOTE NUMBER', 'COMMENT'],
      dtype='object')

In [28]:
# Rename columns (force lower casee and replace spaces with underscore)
df_imp.columns = [clean_column_name(col) for col in df_imp.columns]

df_imp.columns

Index(['type', 'manufacturing_country', 'manufacturer', 'year_of_manufacture',
       'category', 'model_orig', 'model2', 'calibre', 'serial_number',
       'government_user_mark', 'weapon_number', 'batch_lot_number',
       'quantity_in_box_crate', 'expiry_date_', 'box_number', 'out_of_total',
       'end_user_indicated', 'notification_file_name', 'year',
       'exporting_state', 'un_note_number', 'comment'],
      dtype='object')

In [29]:
# Rename specific columns
df_imp = df_imp.rename(columns={
    'serial_number': 'sn',
    'government_user_mark': 'gvtmk',
    'manufacturing_country': 'mfr_country',
    'year_of_manufacture': 'yom',
    'model_orig': 'model'})

df_imp.columns

Index(['type', 'mfr_country', 'manufacturer', 'yom', 'category', 'model',
       'model2', 'calibre', 'sn', 'gvtmk', 'weapon_number', 'batch_lot_number',
       'quantity_in_box_crate', 'expiry_date_', 'box_number', 'out_of_total',
       'end_user_indicated', 'notification_file_name', 'year',
       'exporting_state', 'un_note_number', 'comment'],
      dtype='object')

In [30]:
# Reorganize columns
df_imp = df_imp[[
    'sn', 'gvtmk', 'calibre', 'model',
    'mfr_country', 'category', 'year', 'exporting_state'] + [
        col for col in df_imp.columns if col not in [
            'sn', 'gvtmk', 'calibre', 'model',
    'mfr_country', 'category', 'year', 'exporting_state']
        ]
                ]
df_imp.columns

Index(['sn', 'gvtmk', 'calibre', 'model', 'mfr_country', 'category', 'year',
       'exporting_state', 'type', 'manufacturer', 'yom', 'model2',
       'weapon_number', 'batch_lot_number', 'quantity_in_box_crate',
       'expiry_date_', 'box_number', 'out_of_total', 'end_user_indicated',
       'notification_file_name', 'un_note_number', 'comment'],
      dtype='object')

In [31]:
# Make a df's copy and duplicate specific columns (sufix '_original') to preserve original data
df_imp = df_imp.copy()
columns_to_copy = ['sn', 'year', 'gvtmk', 'category',
                   'model', 'calibre', 'mfr_country', 'exporting_state']
for column in columns_to_copy:
    df_imp[column + '_original'] = df_imp[column]

df_imp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27333 entries, IMP1 to IMP27333
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   sn                        27296 non-null  object 
 1   gvtmk                     3558 non-null   object 
 2   calibre                   27251 non-null  object 
 3   model                     26891 non-null  object 
 4   mfr_country               14167 non-null  object 
 5   category                  27333 non-null  object 
 6   year                      27333 non-null  int64  
 7   exporting_state           27301 non-null  object 
 8   type                      27333 non-null  object 
 9   manufacturer              6367 non-null   object 
 10  yom                       1048 non-null   object 
 11  model2                    24 non-null     object 
 12  weapon_number             20152 non-null  float64
 13  batch_lot_number          1589 non-null   object 
 14  quant

In [ ]:
# Replace null values in specific columns with 'NA'
columns_na = ['sn', 'gvtmk', 'category', 'model', 'calibre', 'mfr_country', 'exporting_state']
for column in columns_na:
    df_imp[column] = df_imp[column].fillna('NA')

# Replace null values in 'year' column with 0
df_imp['year'] = df_imp['year'].fillna(0).astype(int)

df_imp.head(3)

In [33]:
# Attribute string format to columns
columns_to_str = ['sn', 'gvtmk', 'category', 'model', 'calibre', 'mfr_country']
for column in columns_to_str:
    df_imp[column] = df_imp[column].astype(str)

df_imp['sn'].dtype

dtype('O')

**Harmonizing category names**

Acoording to the following table:

|              Original             |     Modified |
|-----------------------------------|--------------|
| Assault rifle                     | assaultrifle |
| Automatic grenade launcher (AGL)  | agl          |
| Explosives                        | explosives   |
| Grenade launchers                 | agl          |
| Heavy machine gun (HMG)           | hmg          |
| Light Machine Gun                 | mg           |
| Medium machine gun                | mg           |
| Medium machine gun (MMG)          | mg           |
| Night vision monocular            | nvm          |
| Self-loading pistol               | pistol       |
| Self-loading rifle                | rifle        |
| Shoulder-fired rocket launcher    | rpg          |

In [34]:
# Check original names
df_imp['category'].value_counts()

category
Assault rifle                       17596
Self-loading rifle                   5721
Self-loading pistol                  2941
Medium machine gun (MMG)              324
Medium machine gun                    272
Heavy machine gun (HMG)               240
Night vision monocular                 70
Shoulder-fired rocket launcher         66
Light Machine Gun                      32
Medium Machine gun (MMG)               30
Automatic grenade launcher (AGL)       12
Grenade launchers                      12
Heavy Machine Gun (HMG)                12
Explosives                              5
Name: count, dtype: int64

In [35]:
# Create a mapping dictionary for category modification
imp_category_mapping = {
    "Assault rifle": "assaultrifle",
    "Automatic grenade launcher (AGL)": "agl",
    "Explosives": "explosives",
    "Grenade launchers": "agl",
    "Heavy machine gun (HMG)": "hmg",
    "Heavy Machine Gun (HMG)": "hmg",
    "Light Machine Gun": "mg",
    "Medium machine gun": "mg",
    "Medium machine gun (MMG)": "mg",
    "Medium Machine gun (MMG)": "mg",
    "Night vision monocular": "nvm",
    "Self-loading pistol": "pistol",
    "Self-loading rifle": "rifle",
    "Shoulder-fired rocket launcher": "rpg"
}

# Replace the 'category' column values based on the mapping
df_imp['category'] = df_imp['category'].map(imp_category_mapping).fillna(df_imp['category'])

# Check changes
df_imp['category'].value_counts()

category
assaultrifle    17596
rifle            5721
pistol           2941
mg                658
hmg               252
nvm                70
rpg                66
agl                24
explosives          5
Name: count, dtype: int64

In [36]:
# Columns to upper
columns_to_upper = ['sn', 'gvtmk', 'mfr_country', 'exporting_state']
for column in columns_to_upper:
    df_imp[column] = df_imp[column].str.upper()

In [ ]:
# Columns to lower
columns_to_lower = ['category', 'model', 'calibre']
for column in columns_to_lower:
    df_imp[column] = df_imp[column].str.lower()

df_imp.head(3)

In [38]:
# Count cells with spaces
count_spaces_per_column = df_imp.map(lambda x: ' ' in str(x)).sum()
print("Number of cells with spaces in each column:")
print(count_spaces_per_column)

Number of cells with spaces in each column:
sn                           2011
gvtmk                        1132
calibre                     27202
model                        9909
mfr_country                   550
category                        0
year                            0
exporting_state              6223
type                            0
manufacturer                 5182
yom                             0
model2                          0
weapon_number                   0
batch_lot_number                3
quantity_in_box_crate           0
expiry_date_                    0
box_number                    108
out_of_total                    0
end_user_indicated          24300
notification_file_name      19200
un_note_number                  0
comment                      4867
sn_original                  2011
year_original                   0
gvtmk_original               1132
category_original           27328
model_original               9909
calibre_original            27202
mfr_

In [ ]:
# Create new columns after cleaning fields (delete spaces, hyphens, dots, etc.) in selected columns, to preserve the originals
columns_to_edit = ['sn', 'category', 'calibre', 'model', 'gvtmk', 'mfr_country', 'exporting_state']

for column in columns_to_edit:
    df_imp[column] = df_imp[column].str.replace(' ', '', regex=True)  # delete spaces
    df_imp[column] = df_imp[column].str.replace('-', '', regex=True) # delete '-'

#Delete '.' in model
df_imp['model'] = df_imp['model'].str.replace('.', '')

#Delete '.' in sn
df_imp['sn'] = df_imp['sn'].str.replace('.', '')

#Delete '.' in 'exporting_state'
df_imp['exporting_state'] = df_imp['exporting_state'].str.replace('.', '')

df_imp.head(3)

In [40]:
# Check if specif 'sn' from original DB was correctly imported (with left 0s)
# iloc[] changes acording to the original DB row
df_imp.iloc[10137]['sn']

'00129'

In [ ]:
# Check
df_imp

# MATCHING

## Copy and prepare  processesd DBs to match

In [42]:
# Copy processed DBs to use in the matching processes
df_documentation = df_doc.copy()

df_importation = df_imp.copy()

In [43]:
# Rename columns in df_documentation with sufix _d
df_documentation.columns = df_documentation.columns + '_d'

# Select a specific subset of columns in df_documentation
df_doctomatch = df_documentation[
    ['sn_d', 'gvtmk_d', 'calibre_d', 'category_d', 'mfr_country_d', 'model_d',
     'year_doc_d', 'yom_d']].copy()

# Define subset df_doctomatch index to correspond to df_documentation index
df_doctomatch.index = df_documentation.index

#Create new columns 'match_level' e 'index_i' with standard values, which
#will be completed after processing.
df_doctomatch['match_level_d'] = 'no_match'
df_doctomatch['index_i'] = 'NA'

# Inspect columns
df_doctomatch.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12462 entries, dbc09958-9d76-4a53-9675-b35e27c0031b to 95ee9174-1d1a-4119-bbf3-59a7e7d44e1a
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   sn_d           12462 non-null  object
 1   gvtmk_d        12462 non-null  object
 2   calibre_d      12462 non-null  object
 3   category_d     12462 non-null  object
 4   mfr_country_d  12462 non-null  object
 5   model_d        12462 non-null  object
 6   year_doc_d     12462 non-null  int32 
 7   yom_d          12462 non-null  int64 
 8   match_level_d  12462 non-null  object
 9   index_i        12462 non-null  object
dtypes: int32(1), int64(1), object(8)
memory usage: 1.3+ MB


In [44]:
# Rename columns in df_importation with sufix _i
df_importation.columns = df_importation.columns + '_i'

# Select subset from df_importation that will be used in cross-referencing process
df_imptomatch = df_importation[['sn_i', 'year_i', 'gvtmk_i', 'model_i',
                                'calibre_i', 'mfr_country_i', 'category_i', 'exporting_state_i']]

#Define subset df_imptomatch index to correspond to df_importation index
df_imptomatch.index = df_imptomatch.index

# Inspect db
df_imptomatch.info()


<class 'pandas.core.frame.DataFrame'>
Index: 27333 entries, IMP1 to IMP27333
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   sn_i               27333 non-null  object
 1   year_i             27333 non-null  int64 
 2   gvtmk_i            27333 non-null  object
 3   model_i            27333 non-null  object
 4   calibre_i          27333 non-null  object
 5   mfr_country_i      27333 non-null  object
 6   category_i         27333 non-null  object
 7   exporting_state_i  27333 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.9+ MB


In [ ]:
# Create a copy of the subsetx 'tomatch'
df_imp_tomatch = df_imptomatch.copy()
df_doc_tomatch = df_doctomatch.copy()

# Check
df_doc_tomatch

# Matches

* SN Perfect Match (A, B)
* SN Partial Match (C, D, E)
* UniqueGovtMark Perfect Match (F)
* SN 4 digits Perfect Match (E)

This is the heart of the matching process, with the different levels created.

In [46]:
# Create a function to check match levels
def check_match_level(row_d):
    global df_imp_tomatch
    sn_d, gvtmk_d, category_d, calibre_d, model_d, year_doc_d, yom_d = row_d['sn_d'], row_d['gvtmk_d'], row_d['category_d'], row_d['calibre_d'], row_d['model_d'], row_d['year_doc_d'], row_d['yom_d']

    for index_i, row_i in df_imp_tomatch.iterrows():
        sn_i, year_i, gvtmk_i, category_i, calibre_i, model_i = row_i['sn_i'], row_i['year_i'], row_i['gvtmk_i'], row_i['category_i'], row_i['calibre_i'], row_i['model_i']

        conditions_met = (
            calibre_d == calibre_i and \
                  category_d == category_i and \
                    year_doc_d >= year_i and \
                        yom_d <= year_i)

        if len(sn_d) >= 6 and sn_d == sn_i:
            if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                return 'A1', index_i
            elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                return 'A2', index_i
            elif (((gvtmk_d == 'NA') and (gvtmk_i != 'NA')) or \
                ((gvtmk_d != 'NA') and (gvtmk_i == 'NA')) or \
                ((gvtmk_d == 'NA') and (gvtmk_i == 'NA'))) and conditions_met:
                    return 'A3', index_i
        elif len(sn_d) == 5 and sn_d == sn_i:
            if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                return 'B1', index_i
            elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                return 'B2', index_i
            elif (((gvtmk_d == 'NA') and (gvtmk_i != 'NA')) or \
                ((gvtmk_d != 'NA') and (gvtmk_i == 'NA')) or \
                ((gvtmk_d == 'NA') and (gvtmk_i == 'NA'))) and conditions_met:
                    return 'B3', index_i
        elif len(sn_d) >= 6 and len(sn_i) == 6 and sn_d[-6:] == sn_i:
            if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                return 'C1', index_i
            elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                return 'C2', index_i
            elif (((gvtmk_d == 'NA') and (gvtmk_i != 'NA')) or \
                ((gvtmk_d != 'NA') and (gvtmk_i == 'NA')) or \
                ((gvtmk_d == 'NA') and (gvtmk_i == 'NA'))) and conditions_met:
                    return 'C3', index_i
        elif len(sn_d) >= 5 and len(sn_i) == 5 and sn_d[-5:] == sn_i:
            if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                return 'D1', index_i
            elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                return 'D2', index_i
            elif (((gvtmk_d == 'NA') and (gvtmk_i != 'NA')) or \
                ((gvtmk_d != 'NA') and (gvtmk_i == 'NA')) or \
                ((gvtmk_d == 'NA') and (gvtmk_i == 'NA'))) and conditions_met:
                    return 'D3', index_i
        elif (model_d == 'vz58' or model_d == 'vz58v' or model_d == 'vz58p'):
            if len(sn_d) == 6 and len(sn_i) == 5 and sn_d[:5] == sn_i:
                if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                    return 'E1', index_i
                elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                    return 'E2', index_i
                elif (((gvtmk_d == 'NA') and (gvtmk_i != 'NA')) or \
                ((gvtmk_d != 'NA') and (gvtmk_i == 'NA')) or \
                ((gvtmk_d == 'NA') and (gvtmk_i == 'NA'))) and conditions_met:
                    return 'E3', index_i
        elif (len(gvtmk_d) >= 11) and (gvtmk_d == gvtmk_i):
            if model_d == model_i and model_d != 'NA' and conditions_met:
                return 'F1', index_i
            elif conditions_met:
                return 'F2', index_i
        elif len(sn_d) == 4 and len(sn_i) == 4 and sn_d == sn_i:
            if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                return 'G1', index_i
            elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                return 'G2', index_i
            elif model_d == model_i and model_d != 'NA' and conditions_met:
                return 'G3', index_i

    return 'no_match', 'NA'

In [47]:
# Apply def check_match_level on the DBs
df_doc_tomatch[['match_level_d', 'index_i']] = df_doc_tomatch.apply(check_match_level, axis=1, result_type='expand')

In [48]:
# Check matches
df_doc_tomatch['match_level_d'].value_counts()

match_level_d
no_match    11434
D3            550
E3            219
D2             84
B3             69
A3             53
A2             30
F2             21
C3              2
Name: count, dtype: int64

In [49]:
# Create a copy of the matechd df
df_doc_tomatch2 = df_doc_tomatch.copy()

In [50]:
# Export first matched df to excel as 'df_doc_tomatch0.xlsx'
df_doc_tomatch.to_excel('df_doc_tomatch0.xlsx', index=True)

In [51]:
# Create a dictionary with excluded indices in df_imp_tomatch which had already matched
excluded_index_i = []

for index, row in df_doc_tomatch2.iterrows():
    # Verifica se o valor de index_i é um índice válido (não 'NA' ou similar)
    if row['index_i'] != 'NA':
        # Adiciona o valor de index_i à lista do dicionário para o índice atual
        excluded_index_i.append(row['index_i'])


In [52]:
# Check
excluded_index_i

['IMP6494',
 'IMP26997',
 'IMP26985',
 'IMP26982',
 'IMP26983',
 'IMP26992',
 'IMP26987',
 'IMP26991',
 'IMP26994',
 'IMP27001',
 'IMP26999',
 'IMP26980',
 'IMP26988',
 'IMP26989',
 'IMP26981',
 'IMP27000',
 'IMP26979',
 'IMP26990',
 'IMP26995',
 'IMP26993',
 'IMP6154',
 'IMP6124',
 'IMP6331',
 'IMP6122',
 'IMP6114',
 'IMP6118',
 'IMP6219',
 'IMP6619',
 'IMP6618',
 'IMP6322',
 'IMP5389',
 'IMP7973',
 'IMP9274',
 'IMP9739',
 'IMP6117',
 'IMP5714',
 'IMP5472',
 'IMP1635',
 'IMP6599',
 'IMP5949',
 'IMP13149',
 'IMP5695',
 'IMP5725',
 'IMP13594',
 'IMP5758',
 'IMP5915',
 'IMP5887',
 'IMP5705',
 'IMP6314',
 'IMP5940',
 'IMP5918',
 'IMP5692',
 'IMP5921',
 'IMP5697',
 'IMP6216',
 'IMP5849',
 'IMP10560',
 'IMP6786',
 'IMP6217',
 'IMP5653',
 'IMP14525',
 'IMP5370',
 'IMP6211',
 'IMP6624',
 'IMP14130',
 'IMP6359',
 'IMP5716',
 'IMP6213',
 'IMP6319',
 'IMP5452',
 'IMP7808',
 'IMP15822',
 'IMP16416',
 'IMP16464',
 'IMP15934',
 'IMP16254',
 'IMP16386',
 'IMP15864',
 'IMP13148',
 'IMP10199',
 'IMP64

In [53]:
# Create df_imp_tomatch_additional excluding rows with index in excluded_index_i
df_imp_tomatch_additional = df_imp_tomatch.drop(excluded_index_i)

In [54]:
# Initialize the columns 'match_level_d_additional' and 'index_i_additional' with default values
df_doc_tomatch2['match_level_d_additional'] = 'no_match'
df_doc_tomatch2['index_i_additional'] = 'NA'

In [55]:
def find_additional_matches(row_d):
    """
    This function finds additional matches for a given row in the documentation dataset
    by comparing it with rows in the import dataset. It returns a match level and the 
    index of the matching row in the import dataset.

    Parameters:
    row_d (pd.Series): A row from the documentation dataset.

    Returns:
    tuple: A tuple containing the match level and the index of the matching row in the import dataset.
    """
    global df_imp_tomatch_additional
    sn_d, gvtmk_d, category_d, calibre_d, model_d, year_doc_d, yom_d = row_d['sn_d'], row_d['gvtmk_d'], row_d['category_d'], row_d['calibre_d'], row_d['model_d'], row_d['year_doc_d'], row_d['yom_d']

    for index_i, row_i in df_imp_tomatch_additional.iterrows():
        sn_i, year_i, gvtmk_i, category_i, calibre_i, model_i = row_i['sn_i'], row_i['year_i'], row_i['gvtmk_i'], row_i['category_i'], row_i['calibre_i'], row_i['model_i']

        # Check if the basic conditions are met
        conditions_met = (
            calibre_d == calibre_i and \
            category_d == category_i and \
            year_doc_d >= year_i and \
            yom_d <= year_i
        )

        # Check for exact serial number match with length >= 6
        if len(sn_d) >= 6 and sn_d == sn_i:
            if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                return 'A1', index_i
            elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                return 'A2', index_i
            elif (((gvtmk_d == 'NA') and (gvtmk_i != 'NA')) or \
                  ((gvtmk_d != 'NA') and (gvtmk_i == 'NA')) or \
                  ((gvtmk_d == 'NA') and (gvtmk_i == 'NA'))) and conditions_met:
                return 'A3', index_i

        # Check for exact serial number match with length == 5
        elif len(sn_d) == 5 and sn_d == sn_i:
            if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                return 'B1', index_i
            elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                return 'B2', index_i
            elif (((gvtmk_d == 'NA') and (gvtmk_i != 'NA')) or \
                  ((gvtmk_d != 'NA') and (gvtmk_i == 'NA')) or \
                  ((gvtmk_d == 'NA') and (gvtmk_i == 'NA'))) and conditions_met:
                return 'B3', index_i

        # Check for partial serial number match with length >= 6
        elif len(sn_d) >= 6 and len(sn_i) == 6 and sn_d[-6:] == sn_i:
            if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                return 'C1', index_i
            elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                return 'C2', index_i
            elif (((gvtmk_d == 'NA') and (gvtmk_i != 'NA')) or \
                  ((gvtmk_d != 'NA') and (gvtmk_i == 'NA')) or \
                  ((gvtmk_d == 'NA') and (gvtmk_i == 'NA'))) and conditions_met:
                return 'C3', index_i

        # Check for partial serial number match with length == 5
        elif len(sn_d) >= 5 and len(sn_i) == 5 and sn_d[-5:] == sn_i:
            if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                return 'D1', index_i
            elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                return 'D2', index_i
            elif (((gvtmk_d == 'NA') and (gvtmk_i != 'NA')) or \
                  ((gvtmk_d != 'NA') and (gvtmk_i == 'NA')) or \
                  ((gvtmk_d == 'NA') and (gvtmk_i == 'NA'))) and conditions_met:
                return 'D3', index_i

        # Check for specific models with partial serial number match
        elif (model_d == 'vz58' or model_d == 'vz58v' or model_d == 'vz58p'):
            if len(sn_d) == 6 and len(sn_i) == 5 and sn_d[:5] == sn_i:
                if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                    return 'E1', index_i
                elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                    return 'E2', index_i
                elif (((gvtmk_d == 'NA') and (gvtmk_i != 'NA')) or \
                      ((gvtmk_d != 'NA') and (gvtmk_i == 'NA')) or \
                      ((gvtmk_d == 'NA') and (gvtmk_i == 'NA'))) and conditions_met:
                    return 'E3', index_i

        # Check for exact government mark match with length >= 11
        elif (len(gvtmk_d) >= 11) and (gvtmk_d == gvtmk_i):
            if model_d == model_i and model_d != 'NA' and conditions_met:
                return 'F1', index_i
            elif conditions_met:
                return 'F2', index_i

        # Check for exact serial number match with length == 4
        elif len(sn_d) == 4 and len(sn_i) == 4 and sn_d == sn_i:
            if gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and model_d == model_i and model_d != 'NA' and conditions_met:
                return 'G1', index_i
            elif gvtmk_d == gvtmk_i and gvtmk_d != 'NA' and conditions_met:
                return 'G2', index_i
            elif model_d == model_i and model_d != 'NA' and conditions_met:
                return 'G3', index_i

    # Return 'no_match' if no conditions are met
    return 'no_match', 'NA'

In [56]:
for index, row in df_doc_tomatch2.iterrows():
    # Call find_additional_matches for the current row
    # Note that df_imp_tomatch_additional no longer contains the excluded indices
    match_level_d_additional, index_i_additional = find_additional_matches(row)
    
    # Update df_doc_tomatch2 with the results
    df_doc_tomatch2.at[index, 'match_level_d_additional'] = match_level_d_additional
    df_doc_tomatch2.at[index, 'index_i_additional'] = index_i_additional
    
    # Update excluded_index_i to include the new match, if any
    if index_i_additional != 'NA':
        excluded_index_i.append(index_i_additional)
        # It is also important to update df_imp_tomatch_additional to reflect this new exclusion
        df_imp_tomatch_additional = df_imp_tomatch_additional.drop(index_i_additional)


In [ ]:
# Check match levels in df_doc_tomatch2
df_doc_tomatch2['match_level_d_additional'].value_counts()

match_level_d_additional
no_match    12377
D3             47
E3             21
B3              9
D2              6
A2              2
Name: count, dtype: int64

In [ ]:
# Count the number of occurrences of each match level in descending order
match_counting = df_doc_tomatch2['match_level_d'].value_counts()
display(match_counting)

match_level_d
no_match    11434
D3            550
E3            219
D2             84
B3             69
A3             53
A2             30
F2             21
C3              2
Name: count, dtype: int64

In [ ]:
def postmatches_model_check(row_d):
    """
    This function checks for errors in the match level for rows in the documentation dataset
    that belong to the 'assaultrifle' category and were exported from 'ETHIOPIA' or 'USA'.
    It iterates over the import dataset to verify if the match level and index should remain unchanged.

    Parameters:
    row_d (pd.Series): A row from the documentation dataset.

    Returns:
    tuple: A tuple containing the match level and the index of the matching row in the import dataset.
           If no conditions are met, it returns ('no_match', 'NA').
    """
    global df_imp_tomatch
    category_d, model_d, match_level_d, index_i = row_d['category_d'], row_d['model_d'], row_d['match_level_d'], row_d['index_i']

    # Iterate over df_imp_tomatch with iterrows()
    for index, row_i in df_imp_tomatch.iterrows():
        model_i, exporting_state_i = row_i['model_i'], row_i['exporting_state_i']

        if (match_level_d not in ['no_match', 'A1', 'B1', 'C1', 'D1', 'F1', 'G1', 'G3']) and \
           (category_d == 'assaultrifle') and \
           (exporting_state_i == 'ETHIOPIA' or exporting_state_i == 'USA') and \
           model_d == model_i:
            # If conditions are met, return the current values unchanged.
            return match_level_d, index_i

    # If no conditions are met, return 'no_match', 'NA'.
    return 'no_match', 'NA'

In [ ]:
# Apply function postmatches_model_check to check for errors in category assaultrifle
df_doc_tomatch2[['match_level_d', 'index_i']] = df_doc_tomatch2.apply(postmatches_model_check, axis=1, result_type='expand')

In [ ]:
def postmatches_model_check_additional(row_d):
    """
    This function checks for errors in the additional match level for rows in the documentation dataset
    that belong to the 'assaultrifle' category and were exported from 'ETHIOPIA' or 'USA'.
    It iterates over the import dataset to verify if the additional match level and index should remain unchanged.

    Parameters:
    row_d (pd.Series): A row from the documentation dataset.

    Returns:
    tuple: A tuple containing the additional match level and the index of the matching row in the import dataset.
           If no conditions are met, it returns ('no_match', 'NA').
    """
    global df_imp_tomatch
    category_d, model_d, match_level_d_additional, index_i_additional = row_d['category_d'], row_d['model_d'], row_d['match_level_d_additional'], row_d['index_i_additional']

    # Iterate over df_imp_tomatch with iterrows()
    for index, row_i in df_imp_tomatch.iterrows():
        model_i, exporting_state_i = row_i['model_i'], row_i['exporting_state_i']

        if (match_level_d_additional not in ['no_match', 'A1', 'B1', 'C1', 'D1', 'F1', 'G1', 'G3']) and \
           (category_d == 'assaultrifle') and \
           (exporting_state_i == 'ETHIOPIA' or exporting_state_i == 'USA') and \
           model_d == model_i:
            # If conditions are met, return the current values unchanged.
            return match_level_d_additional, index_i_additional

    # If no conditions are met, return 'no_match', 'NA'.
    return 'no_match', 'NA'

In [ ]:
# Apply function postmatches_model_check to check for errors in category assaultrifle
df_doc_tomatch2[['match_level_d_additional', 'index_i_additional']] = df_doc_tomatch2.apply(postmatches_model_check, axis=1, result_type='expand')

In [ ]:
# Check df_documentation columns before combining information with df_doctomatch after matching
df_documentation.columns

Index(['sn_d', 'gvtmk_d', 'category_d', 'model_d', 'mfr_country_d',
       'calibre_d', 'year_doc_d', 'yom_d', 'date_doc_d', 'source_d',
       'status_d', 'match_yes_no_d', 'lenght_d', 'security_force_d',
       'security_force2_d', 'region_d', 'district_d',
       'location_police_station_unitunit_d', 'gobolka_degmo_region_district_d',
       '_latitude_d', '_longitude_d', 'other_unique_identifying_mark_d',
       'generic_category_1_d', 'generic_category_3_d', 'qa_note_d',
       'end_user_indicated_d', 'year_imported_d', 'exporting_state_d',
       'un_note_number_d', 'accept_reject_d', 'qa_by_d', 'sn_original_d',
       'gvtmk_original_d', 'category_original_d', 'model_original_d',
       'calibre_original_d', 'date_doc_original_d', 'yom_original_d',
       'mfr_country_original_d'],
      dtype='object')

In [ ]:
# Create a column list, with 'match_level' and 'index_i' at the begining
cols = ['match_level_d', 'index_i', 'match_level_d_additional', 'index_i_additional'] + [
    col for col in df_doc_tomatch2.columns if col not in [
        'match_level_d', 'index_i', 'match_level_d_additional', 'index_i_additional']]

# Reorder the columns of df_a using the modified column list
df_doc_tomatch2 = df_doc_tomatch2[cols]
df_doc_tomatch2

In [ ]:
# Create a new DF merging df_doc_tomatch with df_documentation using the unique reference/key _record_id_
df_documentation_matches = pd.merge(df_doc_tomatch2, df_documentation.drop(
    ['sn_d', 'year_doc_d', 'gvtmk_d', 'calibre_d', 'category_d', 'model_d',
     'yom_d','mfr_country_d', 'security_force_d','security_force2_d',
     'region_d', 'district_d', 'generic_category_1_d','mfr_country_d',
     'end_user_indicated_d', 'sn_original_d', 'gvtmk_original_d',
     'category_original_d', 'model_original_d', 'calibre_original_d',
    'date_doc_original_d'],
    axis=1), left_index=True, right_index=True)

df_documentation_matches.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12462 entries, dbc09958-9d76-4a53-9675-b35e27c0031b to 95ee9174-1d1a-4119-bbf3-59a7e7d44e1a
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   match_level_d                       12462 non-null  object        
 1   index_i                             12462 non-null  object        
 2   match_level_d_additional            12462 non-null  object        
 3   index_i_additional                  12462 non-null  object        
 4   sn_d                                12462 non-null  object        
 5   gvtmk_d                             12462 non-null  object        
 6   calibre_d                           12462 non-null  object        
 7   category_d                          12462 non-null  object        
 8   mfr_country_d                       12462 non-null  object        
 9   model_d                          

In [ ]:
# Display the 'date_doc_d' column from the df_documentation_matches DataFrame
df_documentation_matches['date_doc_d']

In [ ]:
# Prepare df_importation to merge with df_documentation_matches
df_importation_to_merge = df_importation.drop(
    ['weapon_number_i', 'batch_lot_number_i',
     'quantity_in_box_crate_i', 'expiry_date__i', 'box_number_i',
     'category_original_i','model_original_i',
     'calibre_original_i', 'mfr_country_original_i', 'exporting_state_original_i'], axis=1)

df_importation_to_merge['index_importation'] = df_importation_to_merge.index

df_importation_to_merge

In [ ]:
# Perform the first merge using 'index_i'
df_documentation_matches_final = df_documentation_matches.merge(
    df_importation_to_merge, left_on='index_i', right_index=True, how='left')

# Perform the second merge directly on df_documentation_matches_final using 'index_i_additional'
# Use suffixes to differentiate the columns added by the second merge
df_documentation_matches_final = df_documentation_matches_final.merge(
    df_importation_to_merge, left_on='index_i_additional', right_index=True, how='left', suffixes=('', '_additional'))

In [ ]:
# Check df_documentation_matches_final columns
df_documentation_matches_final.columns

Index(['match_level_d', 'index_i', 'match_level_d_additional',
       'index_i_additional', 'sn_d', 'gvtmk_d', 'calibre_d', 'category_d',
       'mfr_country_d', 'model_d', 'year_doc_d', 'yom_d', 'date_doc_d',
       'source_d', 'status_d', 'match_yes_no_d', 'lenght_d',
       'location_police_station_unitunit_d', 'gobolka_degmo_region_district_d',
       '_latitude_d', '_longitude_d', 'other_unique_identifying_mark_d',
       'generic_category_3_d', 'qa_note_d', 'year_imported_d',
       'exporting_state_d', 'un_note_number_d', 'accept_reject_d', 'qa_by_d',
       'yom_original_d', 'mfr_country_original_d', 'sn_i', 'gvtmk_i',
       'calibre_i', 'model_i', 'mfr_country_i', 'category_i', 'year_i',
       'exporting_state_i', 'type_i', 'manufacturer_i', 'yom_i', 'model2_i',
       'out_of_total_i', 'end_user_indicated_i', 'notification_file_name_i',
       'un_note_number_i', 'comment_i', 'sn_original_i', 'year_original_i',
       'gvtmk_original_i', 'index_importation', 'sn_i_additiona

In [ ]:
# Reorder columns
cols = [
    "source_d",
    "status_d",
    "match_level_d",
    "match_level_d_additional",
    "sn_d",
    "gvtmk_d",
    "category_d",
    'mfr_country_d',
    "model_d",
    "calibre_d",
    "year_doc_d",
    "yom_d",
    "yom_original_d",
    "date_doc_d",
    "year_imported_d",
    "exporting_state_d",
    "index_i",
    "sn_i",
    "gvtmk_i",
    "category_i",
    'mfr_country_i',
    "model_i",
    "calibre_i",
    "yom_i",
    "exporting_state_i",
    "year_i",
    "index_i_additional",
    "sn_i_additional",
    "gvtmk_i_additional",
    "category_i_additional",
    'mfr_country_i_additional',
    "model_i_additional",
    "calibre_i_additional",
    "yom_i_additional",
    "exporting_state_i_additional",
    "year_i_additional"
]

df_documentation_matches_final = df_documentation_matches_final[cols]

# Check
df_documentation_matches_final

In [ ]:
# Check final df
df_documentation_matches_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12462 entries, dbc09958-9d76-4a53-9675-b35e27c0031b to 95ee9174-1d1a-4119-bbf3-59a7e7d44e1a
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   source_d                      12462 non-null  object        
 1   status_d                      0 non-null      float64       
 2   match_level_d                 12462 non-null  object        
 3   match_level_d_additional      12462 non-null  object        
 4   sn_d                          12462 non-null  object        
 5   gvtmk_d                       12462 non-null  object        
 6   category_d                    12462 non-null  object        
 7   mfr_country_d                 12462 non-null  object        
 8   model_d                       12462 non-null  object        
 9   calibre_d                     12462 non-null  object        
 10  year_doc_d                    124

In [ ]:
# Export final df to .xlsx
df_documentation_matches_final.to_excel(
    'df_documentation_after_cross_referencing.xlsx', index=True)

In [ ]:
# Export final df to .csv
df_documentation_matches_final.to_csv(
    'df_documentation_after_cross_referencing.csv', index=True)